# Text vectorization

In [3]:
# imports

from gensim.models import Word2Vec
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [4]:
# word2vec model

# Load text
data = pd.read_csv('News_Category_Dataset_IS_course_preprocessed.csv')

# Tokenize text
data['combined_text'] = data['headline'].str.cat(data['short_description'], sep=' ')

tokenized_data = [str(text).split() for text in data['combined_text']]

# Train word2vec model
w2v_model = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, workers=4, epochs=10)

# Function to get average Word2Vec representation for a sentence
def get_average_w2v(tokens):
    vector_sum = 0
    count = 0
    for word in tokens:
        if word in w2v_model.wv:
            vector_sum += w2v_model.wv[word]
            count += 1
    if count != 0:
        return vector_sum / count
    else:
        return [0] * 100  # Return zero vector if no word found

# Add Word2Vec representations to DataFrame
data_w2v = pd.DataFrame([get_average_w2v(text) for text in tokenized_data])
data_w2v.columns = ['w2v_' + str(i) for i in range(100)]
data_w2v = pd.concat([data['category'], data_w2v], axis=1)

# Write to file
data_w2v.to_csv('w2v.csv', index=False)

# Models

In [5]:
# Test a few different classifiers with Word2Vec

# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    data_w2v.drop(columns=['category']), 
    data['category'], 
    test_size=0.2, 
    random_state=42)

# Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
logistic_predictions = logistic_model.predict(X_test)
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
print("Logistic Regression Accuracy:", logistic_accuracy)

# Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

c:\Users\Domen\IS\Inteligentni-sistemi\Inteligentni-sistemi\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.5533380609867288
Random Forest Accuracy: 0.49458008307162393
